<a href="https://colab.research.google.com/github/ion-ian/MachineLearning_Code/blob/master/Neural_Netwrok_1_FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("T4 GPU Enabled")
    except RuntimeError as e:
        print(f"Error enabling memory growth: {e}")
else:
    print("No GPUs available.")


T4 GPU Enabled


In [ ]:
train_dir = r"C:\Users\coder\CNN\train"
test_dir = r"C:\Users\coder\CNN\test"
img_height, img_width = 48, 48
batch_size = 64
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
)
with tf.device('/GPU:0'):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 1)),  # CONV1
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(1024, activation='relu'),
        Dense(train_generator.num_classes, activation='softmax')
    ])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 25
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
model.save("fer2013_cnn_model.h5")


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/25
359/359 [==============================] - 78s 217ms/step - loss: 1.8014 - accuracy: 0.2553 - val_loss: 1.7573 - val_accuracy: 0.2876
Epoch 2/25
359/359 [==============================] - 10s 27ms/step - loss: 1.5445 - accuracy: 0.4010 - val_loss: 1.4968 - val_accuracy: 0.4268
Epoch 3/25
359/359 [==============================] - 10s 28ms/step - loss: 1.3466 - accuracy: 0.4850 - val_loss: 1.3354 - val_accuracy: 0.4842
Epoch 4/25
359/359 [==============================] - 10s 28ms/step - loss: 1.1978 - accuracy: 0.5453 - val_loss: 1.2370 - val_accuracy: 0.5297
Epoch 5/25
359/359 [==============================] - 10s 28ms/step - loss: 1.0572 - accuracy: 0.6028 - val_loss: 1.2376 - val_accuracy: 0.5351
Epoch 6/25
359/359 [==============================] - 10s 28ms/step - loss: 0.9022 - accuracy: 0.6669 - val_loss: 1.2445 - val_accuracy: 0.5570
Epoch 7/

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("fer2013_cnn_model.h5")
print("Model loaded successfully!")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Model loaded successfully!
113/113 [==============================] - 2s 17ms/step - loss: 3.7482 - accuracy: 0.5541
Test Accuracy: 55.41%
